# Lab 06: Error Surfaces & Gradient Descent

**ING3513 - Introduction to Artificial Intelligence and Machine Learning**

In Lab 05, you manually adjusted weights $w_1$, $w_2$ and bias $\beta$ to find a **diagonal decision boundary** that separated Pine from Birch. You discovered that both grain prominence AND brightness matter for classification.

But how did you know **which direction** to move the sliders? You probably used intuition and trial-and-error.

This lab reveals the **systematic approach** that machine learning algorithms use.

**What you'll learn:**

- **Error surfaces** — visualizing how loss changes as you adjust parameters
- **Gradient descent** — the algorithm that "crawls" downhill to find optimal weights
- **Learning rate** — why step size matters (too small = slow, too large = chaos)
- **Local vs global minima** — when algorithms get stuck in the wrong place
- **Why this matters** — every deep learning model you'll ever use trains this way

**The intuition:** Imagine you're blindfolded on a hillside. You want to reach the valley (minimum error). Gradient descent says: "Feel the slope under your feet, and take a step downhill." Repeat until you reach the bottom.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Configure plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

## Part 0: Why This Lab Exists

In Lab 05, you saw that changing weights $w_1, w_2$ and bias $\beta$ moves the decision boundary. You manually tuned these parameters to find the **diagonal line** that separated Pine from Birch — using **both** grain prominence and brightness.

But real machine learning models have **thousands or millions** of parameters. We can't tune them by hand!

**The central question:** How does an algorithm automatically find the best parameters?

**One powerful answer:** Follow the **negative gradient** — the direction of steepest descent on the error surface. This is called **gradient descent**, and it's by far the most widely used optimization method in machine learning.

> **Note:** There are other optimization algorithms (e.g., Newton's method, evolutionary algorithms, closed-form solutions for simple models), but gradient descent dominates modern ML because it scales well to millions of parameters.

Today, you'll:

1. **See** the error surface (a 3D landscape where height = error)
2. **Watch** gradient descent crawl along this surface toward the minimum
3. **Understand** why learning rate and initialization matter
4. **Experience** what happens with non-convex (bumpy) surfaces

Let's continue with the familiar wood dataset from Lab 05.


## Part 1: Generate the Wood Dataset (Review from Lab 05)

We'll use the same wood classification problem from Lab 05: distinguishing **Pine** from **Birch** based on two features:

| Feature              | Symbol | What it measures                                   | Think of it as...                                                  |
| -------------------- | ------ | -------------------------------------------------- | ------------------------------------------------------------------ |
| **Grain Prominence** | `gp`   | Contrast between light and dark bands in the grain | How much the grain pattern "pops" — subtle (0) vs bold stripes (1) |
| **Brightness**       | `b`    | Average lightness of the wood surface              | How light or dark the wood looks overall (0=black, 10=white)       |

The key insight from Lab 05: **both features help separate the classes, but neither alone is enough!**

- **Grain Prominence:** Pine ~0.3, Birch ~0.5 — significant overlap!
- **Brightness:** Pine ~6.0, Birch ~5.0 — significant overlap!

Neither feature alone separates the classes. But **together**, they create a **diagonal** decision boundary — exactly what you discovered in Lab 05!


In [ ]:
# =============================================================================
# STANDARDIZATION HELPERS
# =============================================================================
# Note: In practice, you'd use sklearn.preprocessing.StandardScaler which provides
# fit(), transform(), and inverse_transform() methods. Here we implement our own
# simple version to understand what's happening under the hood.


def standardize(X):
    """Standardize features to mean=0, std=1. Returns (X_scaled, params)."""
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    X_scaled = (X - mean) / std
    return X_scaled, {"mean": mean, "std": std}


def unstandardize(X_scaled, params):
    """Transform standardized features back to original scale."""
    return X_scaled * params["std"] + params["mean"]


def unstandardize_weights(w, beta, params):
    """Transform weights from standardized space to original feature space."""
    w_orig = w / params["std"]
    beta_orig = beta - np.sum(w * params["mean"] / params["std"])
    return w_orig, beta_orig


# =============================================================================
# GENERATE DATA
# =============================================================================
np.random.seed(42)
n_samples = (
    40  # Per class (more than Lab 05's 8, for smoother error surface visualization)
)

# PINE: Lower grain prominence, HIGHER brightness (upper-left in feature space)
pine_gp = np.random.normal(
    0.3, 0.1, n_samples
)  # Grain prominence: centered around 0.3 (overlaps with Birch!)
pine_b = np.random.normal(6.0, 0.8, n_samples)  # Brightness: centered around 6.0

# BIRCH: Higher grain prominence, LOWER brightness (lower-right in feature space)
birch_gp = np.random.normal(
    0.5, 0.1, n_samples
)  # Grain prominence: centered around 0.5 (overlaps with Pine!)
birch_b = np.random.normal(
    5.0, 0.8, n_samples
)  # Brightness: centered around 5.0 (overlaps with Pine!)

# Clip to valid ranges
pine_gp = np.clip(pine_gp, 0, 1)
pine_b = np.clip(pine_b, 0, 10)
birch_gp = np.clip(birch_gp, 0, 1)
birch_b = np.clip(birch_b, 0, 10)

# Combine into arrays: [grain_prominence, brightness]
# Using gp as a₁ and b as a₂ to match Lab 05's feature ordering and notation
X_pine = np.column_stack([pine_gp, pine_b])
X_birch = np.column_stack([birch_gp, birch_b])
X_raw = np.vstack([X_pine, X_birch])

# Labels: -1 for Pine, +1 for Birch
y = np.concatenate([np.ones(n_samples) * -1, np.ones(n_samples)])

# STANDARDIZE features using our helper function
# This is crucial for gradient descent! Without it, the error surface is very elongated.
X, scaler_params = standardize(X_raw)

print(f"Dataset shape: {X.shape} ({2 * n_samples} samples × 2 features)")
print(f"Labels: {np.unique(y)} (-1 = Pine, +1 = Birch)")
print(f"Class balance: {(y == -1).sum()} Pine, {(y == 1).sum()} Birch")

In [ ]:
# Visualize the data (standardized)
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(
    X[y == -1, 0],
    X[y == -1, 1],
    s=100,
    alpha=0.7,
    c="#2E86AB",
    edgecolors="black",
    linewidth=1.5,
    label="Pine",
)
ax.scatter(
    X[y == 1, 0],
    X[y == 1, 1],
    s=100,
    alpha=0.7,
    c="#A23B72",
    edgecolors="black",
    linewidth=1.5,
    label="Birch",
)

ax.set_xlabel("Grain Prominence (standardized)", fontsize=12)
ax.set_ylabel("Brightness (standardized)", fontsize=12)
ax.set_title(
    "Wood Classification Dataset (Standardized Features)",
    fontsize=14,
    fontweight="bold",
)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color="gray", linestyle="--", alpha=0.5)
ax.axvline(x=0, color="gray", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

**Features are now standardized** (mean=0, std=1):

- Pine (blue): lower grain prominence, higher brightness → upper-left
- Birch (pink): higher grain prominence, lower brightness → lower-right

**Why standardize?**

- Original scales were very different (gp: 0-1, b: 0-10)
- This made the error surface very elongated (oval)
- Standardizing makes both features comparable → rounder error surface
- A horizontal line (using brightness only) misclassifies many points
- But a **diagonal line** using **both features** can separate them!


## Part 2: Define the Model and Loss Function

To visualize the error surface in **3D**, we need exactly **2 parameters**. So we'll start with a simple linear model **without bias**:

$$\hat{y} = w_1 a_1 + w_2 a_2$$

where:

- $a_1$ = grain prominence (gp)
- $a_2$ = brightness (b)
- $w_1, w_2$ = weights (parameters to learn)

This is the same perceptron model from Lab 05: $f(\mathbf{w}, \beta) = w_1 \cdot a_1 + w_2 \cdot a_2 + \beta$, but initially without the bias term.

The **decision boundary** is where $\hat{y} = 0$:

$$w_1 a_1 + w_2 a_2 = 0 \quad \Rightarrow \quad a_2 = -\frac{w_1}{w_2} a_1$$

This is a line through the origin with slope $-w_1/w_2$.

**Connection to Lab 05:** In Lab 05, you manually tuned $w_1$, $w_2$, and $\beta$ with sliders to find a diagonal boundary. Now you'll see how gradient descent finds these values automatically!

### The Loss Function: Mean Squared Error (MSE)

We measure how "wrong" our predictions are using **Mean Squared Error**:

$$\text{MSE}(w) = \frac{1}{n}\sum_{i=1}^n(\hat{y}_i - y_i)^2$$

where $\hat{y}_i = w_1 a_{i,1} + w_2 a_{i,2}$ is the model's prediction for sample $i$.

**Key insight:** For any choice of $(w_1, w_2)$, we can compute the MSE. This creates a **surface** where:

- **Horizontal axes:** $w_1$ and $w_2$ (the parameter space)

- **Vertical axis:** MSE (the error)


In [ ]:
# Define the MSE loss function for weights (w1, w2) without bias
def mse_loss(w1, w2, X, y):
    """
    Compute Mean Squared Error for linear model without bias.

    Parameters:
    -----------
    w1, w2 : float
        Weights for the two features
    X : ndarray of shape (n_samples, 2)
        Feature matrix
    y : ndarray of shape (n_samples,)
        True labels

    Returns:
    --------
    loss : float
        Mean squared error
    """
    y_pred = w1 * X[:, 0] + w2 * X[:, 1]  # Linear predictions
    loss = np.mean((y_pred - y) ** 2)
    return loss


# Test it with random weights
w1_test, w2_test = 0.5, 0.3
test_loss = mse_loss(w1_test, w2_test, X, y)
print(f"MSE with w1={w1_test}, w2={w2_test}: {test_loss:.4f}")

In [ ]:
# Visualize the decision boundary for a given set of weights (standardized space)
def plot_decision_boundary(w1, w2, X, y, title="Decision Boundary"):
    """
    Plot the data and decision boundary for given weights.

    Boundary: w1*x1 + w2*x2 = 0  =>  x2 = -(w1/w2)*x1
    """
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot data points
    ax.scatter(
        X[y == -1, 0],
        X[y == -1, 1],
        s=100,
        alpha=0.7,
        c="#2E86AB",
        edgecolors="black",
        linewidth=1.5,
        label="Pine",
    )
    ax.scatter(
        X[y == 1, 0],
        X[y == 1, 1],
        s=100,
        alpha=0.7,
        c="#A23B72",
        edgecolors="black",
        linewidth=1.5,
        label="Birch",
    )

    # Plot decision boundary (if w2 != 0)
    if abs(w2) > 1e-6:
        x1_range = np.array([-3, 3])
        x2_boundary = -(w1 / w2) * x1_range
        ax.plot(x1_range, x2_boundary, "k--", linewidth=2, label="Decision boundary")

    ax.set_xlabel("Grain Prominence (standardized)", fontsize=12)
    ax.set_ylabel("Brightness (standardized)", fontsize=12)
    ax.set_title(title, fontsize=14, fontweight="bold")
    ax.set_xlim([-3, 3])
    ax.set_ylim([-3, 3])
    ax.axhline(y=0, color="gray", linestyle="--", alpha=0.3)
    ax.axvline(x=0, color="gray", linestyle="--", alpha=0.3)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


# Example: Random initialization
w1_init, w2_init = 0.5, 0.5
loss_init = mse_loss(w1_init, w2_init, X, y)
plot_decision_boundary(
    w1_init,
    w2_init,
    X,
    y,
    title=f"Initial Weights: w₁={w1_init}, w₂={w2_init} | MSE={loss_init:.4f}",
)

## Part 3: Compute the Error Surface

Now for the exciting part: let's create a **grid of $(w_1, w_2)$ values** and compute the MSE at each point. This gives us the **error surface**.

> ⚠️ **Important:** We're computing the entire surface here **only for visualization**! In practice, gradient descent **never sees the full surface** — it only computes the gradient at the current position (like feeling the slope under your feet while blindfolded). This is what makes gradient descent scalable to millions of parameters.

**What we'll see:**

- A 3D surface plot (height = MSE)
- A contour plot (top-down view, like a topographic map)

The **minimum** of this surface is where gradient descent wants to go!


In [ ]:
# Create a grid of (w1, w2) values
# With standardized features, weights will be on similar scales
# w1 (grain prominence): positive — higher gp → more likely Birch (+1)
# w2 (brightness): negative — higher brightness → more likely Pine (-1)
w1_range = np.linspace(-3, 3, 100)
w2_range = np.linspace(-3, 3, 100)
W1_grid, W2_grid = np.meshgrid(w1_range, w2_range)

# Compute MSE at each grid point
MSE_grid = np.zeros_like(W1_grid)
for i in range(len(w1_range)):
    for j in range(len(w2_range)):
        MSE_grid[j, i] = mse_loss(W1_grid[j, i], W2_grid[j, i], X, y)

# Find the minimum MSE location
min_idx = np.unravel_index(np.argmin(MSE_grid), MSE_grid.shape)
w1_optimal = W1_grid[min_idx]
w2_optimal = W2_grid[min_idx]
mse_optimal = MSE_grid[min_idx]

print(f"Error surface grid shape: {MSE_grid.shape}")
print(f"MSE range: [{MSE_grid.min():.4f}, {MSE_grid.max():.4f}]")
print(f"Optimal weights (from grid): w₁={w1_optimal:.4f}, w₂={w2_optimal:.4f}")

In [ ]:
# Interactive 3D error surface using Plotly
# You can rotate, zoom, and pan the surface!

fig_3d = go.Figure()

# Add the error surface
fig_3d.add_trace(
    go.Surface(
        x=W1_grid,
        y=W2_grid,
        z=MSE_grid,
        colorscale="Viridis",
        opacity=0.9,
        name="Error Surface",
        showscale=True,
        colorbar=dict(title="MSE", x=1.02),
    )
)

# Mark the minimum point
fig_3d.add_trace(
    go.Scatter3d(
        x=[w1_optimal],
        y=[w2_optimal],
        z=[mse_optimal],
        mode="markers",
        marker=dict(size=10, color="red", symbol="diamond"),
        name=f"Minimum ({w1_optimal:.2f}, {w2_optimal:.2f})",
    )
)

fig_3d.update_layout(
    title=dict(
        text="<b>Interactive 3D Error Surface</b><br><sup>Drag to rotate, scroll to zoom</sup>",
        x=0.5,
    ),
    scene=dict(
        xaxis_title="w₁",
        yaxis_title="w₂",
        zaxis_title="MSE",
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2)),
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, t=60, b=0),
)

fig_3d.show()

# Contour plot (static, for comparison)
fig, ax = plt.subplots(figsize=(10, 8))
contour = ax.contour(W1_grid, W2_grid, MSE_grid, levels=30, cmap="viridis")
ax.clabel(contour, inline=True, fontsize=8)

# Mark the minimum point
ax.scatter(
    w1_optimal,
    w2_optimal,
    color="red",
    s=200,
    marker="*",
    edgecolors="black",
    linewidth=2,
    zorder=5,
)

ax.set_xlabel("w₁", fontsize=12)
ax.set_ylabel("w₂", fontsize=12)
ax.set_title("Contour Plot (Minimum Marked)", fontsize=14, fontweight="bold")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🔍 What You're Seeing

**3D Surface (left):**

- Each point $(w_1, w_2)$ has a "height" = MSE
- The surface is **bowl-shaped** — this is called a **convex** function
- There's a single **global minimum** (the red star)
- Far from the minimum, the error is high (bad predictions)

**Contour Plot (right):**

- Like a topographic map showing "elevation" of error
- Lines closer together = steeper slope
- The center (red star) is the lowest point
- This is where gradient descent will try to reach!

**Connection to Lab 05:** The optimal weights create the **diagonal decision boundary** you discovered manually. Now you can see exactly where that solution lives in parameter space!

**Mini-task:** Look at the contour plot. If you start at $(w_1=0.5, w_2=0.5)$, which direction should you move to reduce error? (Answer: toward the center, downhill!)


## Part 4: Gradient Descent — The Crawling Cursor

Now we'll implement the algorithm that **automatically** finds the minimum.

### How Gradient Descent Works

Remember in Lab 05 when you manually adjusted sliders for $w_1$, $w_2$, and $\beta$ to find the best decision boundary? Gradient descent does this automatically!

Think of yourself blindfolded on a hill. To reach the valley:

1. **Feel the slope** under your feet (compute the gradient)
2. **Take a step downhill** (update weights in the negative gradient direction)
3. **Repeat** until you reach the bottom (or get tired!)

Mathematically:

$$w^{(t+1)} = w^{(t)} - \eta \nabla_w \text{MSE}$$

where:

- $w^{(t)}$ = weights at iteration $t$
- $\eta$ = **learning rate** (step size)
- $\nabla_w \text{MSE}$ = **gradient** (direction of steepest ascent)

### Computing the Gradient (Analytically)

For our MSE loss with $\hat{y} = w_1 a_1 + w_2 a_2$:

$$\nabla_w \text{MSE} = \frac{2}{n} A^T (Aw - y)$$

where $A$ is the feature matrix and $w = [w_1, w_2]^T$.

This tells us how MSE changes as we tweak $w_1$ and $w_2$.


In [ ]:
def compute_gradient(w, X, y):
    """
    Compute the gradient of MSE with respect to weights w.

    Parameters:
    -----------
    w : ndarray of shape (2,)
        Current weights [w1, w2]
    X : ndarray of shape (n_samples, 2)
        Feature matrix
    y : ndarray of shape (n_samples,)
        True labels

    Returns:
    --------
    gradient : ndarray of shape (2,)
        Gradient vector [∂MSE/∂w1, ∂MSE/∂w2]
    """
    n = len(y)
    y_pred = X @ w  # Matrix multiplication: X·w
    gradient = (2 / n) * X.T @ (y_pred - y)
    return gradient


# Test the gradient function
w_test = np.array([0.5, 0.3])
grad_test = compute_gradient(w_test, X, y)
print(f"Gradient at w=[0.5, 0.3]: {grad_test}")
print("Interpretation: To reduce MSE, move in the OPPOSITE direction of the gradient.")

In [ ]:
def gradient_descent(X, y, w_init, learning_rate=0.01, epochs=100):
    """
    Perform gradient descent to find optimal weights.

    Parameters:
    -----------
    X : ndarray of shape (n_samples, 2)
        Feature matrix
    y : ndarray of shape (n_samples,)
        True labels
    w_init : ndarray of shape (2,)
        Initial weights [w1, w2]
    learning_rate : float
        Step size for each update
    epochs : int
        Number of iterations

    Returns:
    --------
    w_history : ndarray of shape (epochs+1, 2)
        Weights at each iteration (including initial)
    loss_history : ndarray of shape (epochs+1,)
        MSE at each iteration (including initial)
    """
    w = w_init.copy()
    w_history = [w.copy()]
    loss_history = [mse_loss(w[0], w[1], X, y)]

    for epoch in range(epochs):
        # Compute gradient
        grad = compute_gradient(w, X, y)

        # Update weights (move in negative gradient direction)
        w = w - learning_rate * grad

        # Record history
        w_history.append(w.copy())
        loss_history.append(mse_loss(w[0], w[1], X, y))

    return np.array(w_history), np.array(loss_history)


# Run gradient descent from a random starting point
w_init = np.array([0.5, 0.5])
learning_rate = 0.1
epochs = 50

w_history, loss_history = gradient_descent(X, y, w_init, learning_rate, epochs)

print(f"Initial weights: {w_init}")
print(f"Final weights: {w_history[-1]}")
print(f"Initial MSE: {loss_history[0]:.4f}")
print(f"Final MSE: {loss_history[-1]:.4f}")
print(f"Improvement: {loss_history[0] - loss_history[-1]:.4f}")

In [ ]:
# Visualize the optimization process

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Left: Loss vs Epoch
axes[0].plot(loss_history, linewidth=2, color="#2E86AB")
axes[0].scatter(0, loss_history[0], s=100, color="red", zorder=5, label="Start")
axes[0].scatter(
    len(loss_history) - 1, loss_history[-1], s=100, color="green", zorder=5, label="End"
)
axes[0].set_xlabel("Epoch", fontsize=12)
axes[0].set_ylabel("MSE", fontsize=12)
axes[0].set_title("Loss Decreases Over Time", fontsize=14, fontweight="bold")
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Middle: Path on contour plot
contour = axes[1].contour(
    W1_grid, W2_grid, MSE_grid, levels=20, cmap="viridis", alpha=0.6
)
axes[1].clabel(contour, inline=True, fontsize=8)

# Plot the gradient descent path
axes[1].plot(
    w_history[:, 0],
    w_history[:, 1],
    "o-",
    color="red",
    linewidth=2,
    markersize=6,
    label="GD path",
)
axes[1].scatter(
    w_history[0, 0],
    w_history[0, 1],
    s=200,
    color="red",
    marker="o",
    edgecolors="black",
    linewidth=2,
    label="Start",
    zorder=5,
)
axes[1].scatter(
    w_history[-1, 0],
    w_history[-1, 1],
    s=200,
    color="green",
    marker="*",
    edgecolors="black",
    linewidth=2,
    label="End",
    zorder=5,
)
axes[1].scatter(
    w1_optimal,
    w2_optimal,
    s=200,
    color="gold",
    marker="X",
    edgecolors="black",
    linewidth=2,
    label="True minimum",
    zorder=5,
)

axes[1].set_xlabel("w₁", fontsize=12)
axes[1].set_ylabel("w₂", fontsize=12)
axes[1].set_title(
    'Gradient Descent Path (The "Crawling Cursor")', fontsize=14, fontweight="bold"
)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# Right: Decision boundary with learned weights
axes[2].scatter(
    X[y == -1, 0],
    X[y == -1, 1],
    s=100,
    alpha=0.7,
    c="#2E86AB",
    edgecolors="black",
    linewidth=1.5,
    label="Pine",
)
axes[2].scatter(
    X[y == 1, 0],
    X[y == 1, 1],
    s=100,
    alpha=0.7,
    c="#A23B72",
    edgecolors="black",
    linewidth=1.5,
    label="Birch",
)

# Plot decision boundary for final weights (no bias: w1*x1 + w2*x2 = 0)
w1_final, w2_final = w_history[-1]
if abs(w2_final) > 1e-6:
    x1_range = np.array([-3, 3])
    x2_boundary = -(w1_final / w2_final) * x1_range
    axes[2].plot(x1_range, x2_boundary, "k--", linewidth=2.5, label="Learned boundary")

axes[2].set_xlabel("Grain Prominence (standardized)", fontsize=12)
axes[2].set_ylabel("Brightness (standardized)", fontsize=12)
axes[2].set_title(
    f"Decision Boundary: w₁={w1_final:.2f}, w₂={w2_final:.2f}",
    fontsize=14,
    fontweight="bold",
)
axes[2].set_xlim([-3, 3])
axes[2].set_ylim([-3, 3])
axes[2].axhline(y=0, color="gray", linestyle="--", alpha=0.3)
axes[2].axvline(x=0, color="gray", linestyle="--", alpha=0.3)
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🔍 What You're Seeing

**Left plot (Loss vs Epoch):** The MSE decreases over time — the algorithm is learning!

**Middle plot (Gradient Descent Path):** The red dots trace the path gradient descent takes through parameter space. Notice:

- It starts at the red circle (initial guess)
- It follows the contour lines downhill
- It converges near the gold X (true minimum)

**Right plot (Decision Boundary):** The learned weights create a diagonal decision boundary that separates Pine from Birch — this is the best boundary gradient descent can find with these parameters!

**Key insight:** Gradient descent is like a ball rolling downhill. It naturally finds low points on the error surface.

**Connection to Lab 05:** Remember manually adjusting $w_1$, $w_2$, and $\beta$ with the interactive sliders until the line separated Pine from Birch? Gradient descent automates exactly that process — it computes which direction reduces error and takes a step that way!

**Mini-task:** Try running the cell above with different starting points:

- `w_init = np.array([-0.5, 2.0])`
- `w_init = np.array([0.8, 0.2])`

Does gradient descent still find the minimum? (Hint: Yes, because this surface is convex!)


## Part 5: Learning Rate & Epochs — The Goldilocks Problem

Gradient descent has two critical hyperparameters:

1. **Learning rate ($\eta$):** How big a step to take
2. **Epochs:** How many steps to take

Let's see what happens when these are **too small**, **too large**, or **just right**.

### The 4 Experiments:

1. **Good learning rate + enough epochs** → converges smoothly ✓
2. **Too small learning rate** → slow, looks stuck 🐌
3. **Too large learning rate** → diverges or oscillates 💥
4. **Good learning rate + too few epochs** → stops early ⏱️


In [ ]:
# Run 4 experiments with different settings
experiments = [
    {"lr": 0.1, "epochs": 50, "label": "Good LR + Enough Epochs"},
    {"lr": 0.01, "epochs": 50, "label": "Too Small LR (slow)"},
    {"lr": 0.7, "epochs": 10, "label": "Too Large LR (unstable)"},
    {"lr": 0.1, "epochs": 10, "label": "Good LR + Too Few Epochs"},
]

# Initial point for all experiments
w_init = np.array([0.6, 0.3])

# Run all experiments
results = []
for exp in experiments:
    w_hist, loss_hist = gradient_descent(X, y, w_init, exp["lr"], exp["epochs"])
    results.append(
        {
            "label": exp["label"],
            "w_history": w_hist,
            "loss_history": loss_hist,
            "final_loss": loss_hist[-1],
            "converged": loss_hist[-1] < 0.1,  # Threshold for "converged"
        }
    )

# Print summary table
print("Experiment Results Summary")
print("=" * 70)
print(f"{'Experiment':<35} {'Final Loss':>12} {'Converged?':>12}")
print("-" * 70)
for res in results:
    status = "✓" if res["converged"] else "✗"
    print(f"{res['label']:<35} {res['final_loss']:>12.4f} {status:>12}")

In [ ]:
# Visualize all 4 experiments in a 2×2 grid

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, res in enumerate(results):
    ax = axes[idx]

    # Plot contour
    contour = ax.contour(
        W1_grid, W2_grid, MSE_grid, levels=20, cmap="viridis", alpha=0.4
    )

    # Plot path
    w_hist = res["w_history"]
    ax.plot(
        w_hist[:, 0],
        w_hist[:, 1],
        "o-",
        color="red",
        linewidth=2,
        markersize=5,
        alpha=0.8,
    )
    ax.scatter(
        w_hist[0, 0],
        w_hist[0, 1],
        s=200,
        color="red",
        marker="o",
        edgecolors="black",
        linewidth=2,
        zorder=5,
    )
    ax.scatter(
        w_hist[-1, 0],
        w_hist[-1, 1],
        s=200,
        color="green",
        marker="*",
        edgecolors="black",
        linewidth=2,
        zorder=5,
    )
    ax.scatter(
        w1_optimal,
        w2_optimal,
        s=150,
        color="gold",
        marker="X",
        edgecolors="black",
        linewidth=2,
        zorder=4,
    )

    ax.set_xlabel("w₁", fontsize=11)
    ax.set_ylabel("w₂", fontsize=11)
    ax.set_title(
        f"{res['label']}\nFinal Loss: {res['final_loss']:.4f}",
        fontsize=12,
        fontweight="bold",
    )
    ax.grid(True, alpha=0.3)
    ax.set_xlim([W1_grid.min(), W1_grid.max()])
    ax.set_ylim([W2_grid.min(), W2_grid.max()])

plt.tight_layout()
plt.show()

Notice how learning rate affects the **speed** and **stability** of convergence!


### 🔍 What You're Seeing

**Top-left (Good LR + Enough Epochs):**

- Smooth path toward the minimum
- Converges close to the optimal point ✓

**Top-right (Too Small LR):**

- Takes tiny steps (slow progress)
- May not reach the minimum within the given epochs 🐌

**Bottom-left (Too Large LR):**

- Takes huge steps that overshoot
- May oscillate or even diverge (bounce around) 💥
- Sometimes still converges, but inefficiently

**Bottom-right (Good LR + Too Few Epochs):**

- Moving in the right direction but stops too early ⏱️
- Needs more iterations to reach the minimum

**The moral:** Choosing the right learning rate is crucial! Too small = slow, too large = chaos, just right = efficient convergence.

**Mini-task:** Add a 5th experiment with an extremely large learning rate (e.g., `lr=1.0`). What happens?


In [ ]:
# Comparison of loss curves for all experiments

fig, ax = plt.subplots(figsize=(12, 6))

colors = ["green", "blue", "red", "orange"]
for idx, res in enumerate(results):
    ax.plot(
        res["loss_history"],
        linewidth=2,
        color=colors[idx],
        label=res["label"],
        marker="o",
        markersize=4,
        alpha=0.8,
    )

ax.set_xlabel("Epoch", fontsize=12)
ax.set_ylabel("MSE", fontsize=12)
ax.set_title(
    "Loss Curves for Different Hyperparameter Settings", fontsize=14, fontweight="bold"
)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_yscale("log")  # Log scale to see details for small losses

plt.tight_layout()
plt.show()

## Part 6: Non-Convex Surfaces — When Things Get Tricky

So far, our error surface has been **convex** (bowl-shaped) with a single minimum. Gradient descent easily finds it from any starting point.

But many real ML problems have **non-convex** loss surfaces with:

- **Multiple local minima** — valleys that aren't the lowest point
- **Saddle points** — flat regions where gradient ≈ 0
- **Plateaus** — areas where learning gets stuck

Let's explore this using a classic non-convex function: **Himmelblau's function**.

$$f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2$$

This function has **4 local minima** of equal depth — a perfect toy example!


In [ ]:
# Define Himmelblau's function (a classic non-convex test function)
def himmelblau(x, y):
    """
    Himmelblau's function: has 4 local minima of equal depth.

    Minima at approximately:
    - (3.0, 2.0)
    - (-2.805, 3.131)
    - (-3.779, -3.283)
    - (3.584, -1.848)
    """
    return (x**2 + y - 11) ** 2 + (x + y**2 - 7) ** 2


# Gradient of Himmelblau's function (computed analytically)
def himmelblau_gradient(x, y):
    """Compute gradient of Himmelblau's function."""
    df_dx = 4 * x * (x**2 + y - 11) + 2 * (x + y**2 - 7)
    df_dy = 2 * (x**2 + y - 11) + 4 * y * (x + y**2 - 7)
    return np.array([df_dx, df_dy])


# Create a grid for visualization
x_range = np.linspace(-5, 5, 200)
y_range = np.linspace(-5, 5, 200)
X_grid, Y_grid = np.meshgrid(x_range, y_range)
Z_grid = himmelblau(X_grid, Y_grid)

print("Himmelblau's function computed on 200×200 grid")
print(f"Function value range: [{Z_grid.min():.4f}, {Z_grid.max():.4f}]")

In [ ]:
# Visualize Himmelblau's function - Interactive 3D with Plotly

# The 4 minima locations
minima = [(3.0, 2.0), (-2.805, 3.131), (-3.779, -3.283), (3.584, -1.848)]

# Interactive 3D surface using Plotly
fig_3d_himmel = go.Figure()

# Add the surface
fig_3d_himmel.add_trace(
    go.Surface(
        x=X_grid,
        y=Y_grid,
        z=Z_grid,
        colorscale="Viridis",
        opacity=0.9,
        name="Himmelblau Surface",
        showscale=True,
        colorbar=dict(title="f(x,y)", x=1.02),
    )
)

# Mark the 4 minima
for i, (xmin, ymin) in enumerate(minima):
    zmin = himmelblau(xmin, ymin)
    fig_3d_himmel.add_trace(
        go.Scatter3d(
            x=[xmin],
            y=[ymin],
            z=[zmin],
            mode="markers",
            marker=dict(size=8, color="red", symbol="diamond"),
            name=f"Min {i + 1}: ({xmin:.2f}, {ymin:.2f})",
        )
    )

fig_3d_himmel.update_layout(
    title=dict(
        text="<b>Himmelblau's Function (Non-Convex)</b><br><sup>Drag to rotate, scroll to zoom — notice the 4 valleys!</sup>",
        x=0.5,
    ),
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="f(x, y)",
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2)),
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, t=60, b=0),
)

fig_3d_himmel.show()

# Contour plot (static, for comparison)
fig, ax = plt.subplots(figsize=(10, 8))
contour = ax.contour(X_grid, Y_grid, Z_grid, levels=30, cmap="viridis")
ax.clabel(contour, inline=True, fontsize=8)

# Mark the 4 minima
for xmin, ymin in minima:
    ax.scatter(
        xmin,
        ymin,
        color="red",
        s=200,
        marker="*",
        edgecolors="black",
        linewidth=2,
        zorder=5,
    )

ax.set_xlabel("x", fontsize=12)
ax.set_ylabel("y", fontsize=12)
ax.set_title("Contour Plot (4 Minima Marked)", fontsize=14, fontweight="bold")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Gradient descent on Himmelblau's function
def gradient_descent_himmelblau(x_init, y_init, learning_rate=0.01, epochs=100):
    """
    Gradient descent on Himmelblau's function.

    Returns:
    --------
    history : ndarray of shape (epochs+1, 2)
        [x, y] at each iteration
    loss_history : ndarray of shape (epochs+1,)
        Function value at each iteration
    """
    pos = np.array([x_init, y_init])
    history = [pos.copy()]
    loss_history = [himmelblau(pos[0], pos[1])]

    for epoch in range(epochs):
        grad = himmelblau_gradient(pos[0], pos[1])
        pos = pos - learning_rate * grad

        history.append(pos.copy())
        loss_history.append(himmelblau(pos[0], pos[1]))

    return np.array(history), np.array(loss_history)


# Try gradient descent from 6 different random starting points
np.random.seed(123)
n_trials = 6
starting_points = np.random.uniform(-4, 4, size=(n_trials, 2))

# Run gradient descent from each starting point
results_himmel = []
for i, start in enumerate(starting_points):
    history, loss_history = gradient_descent_himmelblau(
        start[0], start[1], learning_rate=0.01, epochs=200
    )
    results_himmel.append(
        {
            "start": start,
            "history": history,
            "loss_history": loss_history,
            "final": history[-1],
            "final_loss": loss_history[-1],
        }
    )

print(f"Ran gradient descent from {n_trials} random starting points")
print("\nResults:")
print("=" * 70)
for i, res in enumerate(results_himmel):
    print(
        f"Trial {i + 1}: Start=({res['start'][0]:.2f}, {res['start'][1]:.2f}) "
        f"→ End=({res['final'][0]:.2f}, {res['final'][1]:.2f}), "
        f"Loss={res['final_loss']:.4f}"
    )

Notice the 4 "valleys" — each is a **local minimum**! Depending on where you start, gradient descent will find different minima.


In [ ]:
# Visualize all gradient descent paths on the contour plot

fig, ax = plt.subplots(figsize=(12, 10))

# Plot contour
contour = ax.contour(X_grid, Y_grid, Z_grid, levels=30, cmap="viridis", alpha=0.5)
ax.clabel(contour, inline=True, fontsize=8)

# Mark the 4 true minima
for xmin, ymin in minima:
    ax.scatter(
        xmin,
        ymin,
        color="gold",
        s=300,
        marker="*",
        edgecolors="black",
        linewidth=2,
        zorder=5,
        label="True minima",
    )

# Plot each gradient descent path
colors = ["red", "blue", "green", "purple", "orange", "brown"]
for i, res in enumerate(results_himmel):
    hist = res["history"]
    ax.plot(
        hist[:, 0],
        hist[:, 1],
        "o-",
        color=colors[i],
        linewidth=2,
        markersize=4,
        alpha=0.8,
        label=f"Trial {i + 1}",
    )
    ax.scatter(
        hist[0, 0],
        hist[0, 1],
        s=150,
        color=colors[i],
        marker="o",
        edgecolors="black",
        linewidth=2,
        zorder=4,
    )
    ax.scatter(
        hist[-1, 0],
        hist[-1, 1],
        s=150,
        color=colors[i],
        marker="X",
        edgecolors="black",
        linewidth=2,
        zorder=4,
    )

# Remove duplicate labels for minima
handles, labels = ax.get_legend_handles_labels()
unique_labels = {}
for handle, label in zip(handles, labels):
    if label not in unique_labels:
        unique_labels[label] = handle
ax.legend(unique_labels.values(), unique_labels.keys(), fontsize=10, loc="upper left")

ax.set_xlabel("x", fontsize=12)
ax.set_ylabel("y", fontsize=12)
ax.set_title(
    "Gradient Descent from Multiple Starting Points\n(Different starts → Different minima!)",
    fontsize=14,
    fontweight="bold",
)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

See how different starting points lead to different local minima? This is the challenge of **non-convex optimization**!


### 🔍 The Local Minima Problem

**What you're seeing:**

- Gradient descent gets trapped in whichever minimum is **closest to the starting point**
- All 4 minima are equally good (same function value)
- But in real ML, some local minima are better than others!

**Why this matters:**

- In deep learning, loss surfaces are highly non-convex
- Random initialization can lead to different (and sometimes bad) solutions
- This is why we often train models multiple times with different seeds

**Mitigation strategies:**

1. **Random restarts** — try many initializations, pick the best result
2. **Better initialization** — use smart starting points (e.g., Xavier, He initialization)
3. **Momentum** — help gradient descent "roll through" small bumps
4. **Adaptive learning rates** — algorithms like Adam that adjust step size automatically
5. **Stochastic gradient descent** — noise from mini-batches can help escape shallow local minima


### Mini-Demo: Random Restarts

Since gradient descent gets trapped in the **nearest** minimum, one practical solution is to try **many random starting points** and keep the best result!

This simple strategy is widely used in practice.


In [ ]:
# Random restarts: try many starting points, keep the best
np.random.seed(42)
n_restarts = 20

restart_results = []
for i in range(n_restarts):
    # Random starting point
    start = np.random.uniform(-4, 4, size=2)

    # Run gradient descent
    history, loss_history = gradient_descent_himmelblau(
        start[0], start[1], learning_rate=0.01, epochs=200
    )

    restart_results.append(
        {
            "start": start,
            "history": history,
            "final": history[-1],
            "final_loss": loss_history[-1],
        }
    )

# Find the best result
best_idx = np.argmin([r["final_loss"] for r in restart_results])
best_result = restart_results[best_idx]

# Count how many times we found each minimum
print(f"Random Restarts: {n_restarts} attempts")
print("=" * 70)
print("\nFinal positions (which minimum each run found):")
for i, res in enumerate(restart_results):
    print(
        f"  Run {i + 1:2d}: ({res['final'][0]:6.2f}, {res['final'][1]:6.2f}) → loss = {res['final_loss']:.6f}"
    )

print(f"\nBest result: Run {best_idx + 1}")
print(f"  Final: ({best_result['final'][0]:.3f}, {best_result['final'][1]:.3f})")
print(f"  Loss: {best_result['final_loss']:.6f}")

In [ ]:
# Visualize all random restart paths

fig, ax = plt.subplots(figsize=(12, 10))

# Plot contour
contour = ax.contour(X_grid, Y_grid, Z_grid, levels=30, cmap="viridis", alpha=0.4)

# Mark the 4 true minima
for xmin, ymin in minima:
    ax.scatter(
        xmin,
        ymin,
        color="gold",
        s=400,
        marker="*",
        edgecolors="black",
        linewidth=2,
        zorder=10,
    )

# Plot all paths (thin gray lines)
for res in restart_results:
    ax.plot(
        res["history"][:, 0], res["history"][:, 1], "gray", linewidth=0.8, alpha=0.4
    )
    # Starting points (small circles)
    ax.scatter(
        res["start"][0], res["start"][1], s=30, color="blue", alpha=0.5, zorder=4
    )
    # Final points (small X markers)
    ax.scatter(
        res["final"][0],
        res["final"][1],
        s=50,
        color="red",
        marker="x",
        alpha=0.7,
        zorder=5,
    )

# Highlight the best path
ax.plot(
    best_result["history"][:, 0],
    best_result["history"][:, 1],
    "green",
    linewidth=3,
    alpha=0.9,
    label="Best run",
)
ax.scatter(
    best_result["start"][0],
    best_result["start"][1],
    s=200,
    color="green",
    marker="o",
    edgecolors="black",
    linewidth=2,
    zorder=8,
    label="Best start",
)
ax.scatter(
    best_result["final"][0],
    best_result["final"][1],
    s=200,
    color="green",
    marker="X",
    edgecolors="black",
    linewidth=2,
    zorder=8,
    label="Best final",
)

ax.set_xlabel("x", fontsize=12)
ax.set_ylabel("y", fontsize=12)
ax.set_title(
    f"Random Restarts: {n_restarts} Attempts\n(Different starts → Different minima → Pick the best!)",
    fontsize=14,
    fontweight="bold",
)
ax.legend(fontsize=11, loc="upper left")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Key observations:**

- Different starting points land in different local minima
- By trying many starts, we explore the whole landscape
- Pick the best result → more likely to find a good solution!

💡 **This is a real technique used in practice!** In deep learning, we often train multiple models with different random seeds.


## Part 7: Wrap-Up — Connecting the Dots

### What You Learned Today

1. **Error surfaces** visualize how loss changes with parameters
   - Convex surfaces (bowl-shaped) have a single minimum
   - Non-convex surfaces have multiple local minima and saddle points

2. **Gradient descent** is the fundamental optimization algorithm
   - Follow the gradient (direction of steepest descent) to find minima
   - Learning rate controls step size (too small = slow, too large = unstable)
   - Requires enough epochs to converge

3. **Hyperparameters matter**
   - Learning rate and number of epochs critically affect performance
   - Need to tune these for each problem

4. **Non-convex problems are tricky**
   - Different starting points → different solutions
   - Random restarts and smart initialization help
   - This is a fundamental challenge in deep learning

### The Red Thread from Lab 05

In Lab 05, you **manually** adjusted weights $w_1, w_2, \beta$ to find the **diagonal decision boundary** separating Pine from Birch. You learned:

- $w_1$ (grain prominence) and $w_2$ (brightness) control the **slope** of the boundary
- $\beta$ (bias) controls the **position** of the boundary
- Both features matter — that's why the boundary is diagonal!

Today, you saw how **gradient descent automatically** discovers these same optimal parameters by:

- Computing the gradient (which direction reduces error)
- Taking steps in that direction
- Repeating until convergence

**The key connection:** Those sliders you adjusted in Lab 05? Gradient descent moves them for you, following the steepest downhill path on the error surface.

**This is how ALL modern machine learning models learn:** from linear regression to GPT!
